In [1]:
!nvidia-smi

Tue May 11 19:15:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3
!pip3 install gputil
!pip3 install tabulate

     |████████████████████████████████| 163kB 9.2MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 3.3MB 30.0MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=18559da1f501c1ebd5f04350278e41e77a5f60653bbe56e860fcc63a75591a3a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import pandas as pd
import re
import os
data = pd.read_csv('drive/MyDrive/dialogues.tsv', sep='\t')
data = data.drop(9635)
data = data.reset_index(drop=True)

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import cv2
cv2.setNumThreads(0)

import argparse
from google.colab import drive



if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print('GPU')
else:
  device = torch.device("cpu")

GPU


In [7]:

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2').to(device)
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')



/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [8]:
SPECIAL_TOKENS = {'bos_token' : "<bos>", "eos_token" :"<eos>", 'additional_special_tokens': ["<speaker1>", "<speaker2>", "<fos>"]}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

Embedding(50263, 1024)

In [9]:
def merge_replyes(buffer):
    history = []
    i = 0
    a = 0
    b = a
    while (i < len(buffer)):
        same = ''
        a = i
        b = a
        while (i < len(buffer) and b < len(buffer) and buffer[a][0] == buffer[b][0]):
            b += 1
        
        for k in range(a, b):
            same += buffer[k][1] + '. '
        history.append([buffer[a][0], same])
        i = b
    return history

In [10]:
import re
import random
def generate_example(row):
    parsed = re.findall(r'Пользователь [12].+?(?=</span>)', data['dialogue'][row].replace("<br />", " "))
    history = merge_replyes([[1 if sentenses[13] == '1' else 2, sentenses[16:]] for sentenses in parsed])
    # print(history)
    random_slice = random.randint(1, min(len(history) - 1, 18))   # MAKE MAX 20 SENTENSES
    
    if history[random_slice][0] == 1:
        persona = re.findall(r'.+?(?=</span>)', data['persona_2_profile'][row].replace("<br />", " "))[0][26:]
    else:
        persona = re.findall(r'.+?(?=</span>)', data['persona_1_profile'][row].replace("<br />", " "))[0][26:]
        
    reply = history[random_slice][1]
    history = [[i[1]] for i in history[:random_slice]]
    
    
    return history, [reply], persona.split()
    

In [11]:
generate_example(100)

([['Привет, друг мой!. Как дела, как жизнь?. Знакомиться будем?. '],
  ['добрый вечер!. конечно, будем, это легально )) как юрист заявляю вам. как у вас дела?. '],
  ['Ого! Юрист! Наверное столичный?))). Да вот все супер! Наворачиваю блины с икрой, радуюсь жизни. '],
  ['есть такое) мой папа бизнесмен, а кто то же должен знать юридические тонкости. а кем ты работаешь?. мммм блины! а ананасы любишь? я вот обожаю. ']],
 ['Завидую белой завистью! А то я сам из провинции, пора бы тоже в столицу! Работа, кстати, у меня интересная .. я биолог .. правда в школе, с сорванцами эими. этими*. да, ананасы это круто !. '],
 ['Я',
  'юрист.',
  'Мой',
  'папа',
  'бизнесмен.',
  'Я',
  'верю',
  'в',
  'любовь.',
  'Я',
  'люблю',
  'ананас.',
  'У',
  'меня',
  'есть',
  'кошка.'])

In [12]:
from itertools import chain

persona = [["Я", "люблю", "играть", "в", "баскетбол", "."],
           ["Я", "студент", "из", "Москвы", "."]]

history = [["привет", "как", "Дела", "?"],
           ["Все", "хорошо", "спасибо", "."]]

reply = ["Рад", "это", "слышать"]

bos, eos, speaker1, speaker2, fos = "<bos>", "<eos>", "<speaker1>", "<speaker2>", "<fos>"

def get_segments(words):
  mask = []
  i = len(words) - 1
  while words[i] != speaker1 and words[i] != speaker2:
    i -= 1
  fst_token = words[i]

  if fst_token == speaker1:
    fill_token = speaker1
  else:
    fill_token = speaker2

  for i in range(len(words)):
    if words[i] == speaker2 and fill_token == speaker1:
      fill_token = speaker2

    if words[i] == speaker1 and fill_token == speaker2:
      fill_token = speaker1

    mask.append(fill_token)
  return mask

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    sequence = [[bos] + list(chain(*persona))] + history +  [[fos] + reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

words, segments, position, sequence = build_inputs(persona, history, reply)
print(words)
print(segments)

['<bos>', 'Я', 'люблю', 'играть', 'в', 'баскетбол', '.', 'Я', 'студент', 'из', 'Москвы', '.', '<speaker1>', 'привет', 'как', 'Дела', '?', '<speaker2>', 'Все', 'хорошо', 'спасибо', '.', '<speaker1>', '<fos>', 'Рад', 'это', 'слышать', '<eos>']
['<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker1>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker2>', '<speaker2>']


In [13]:
get_segments(words)

['<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>']

In [14]:
def get_segments(words):
  speaker1 = tokenizer.convert_tokens_to_ids('<speaker1>')
  speaker2 = tokenizer.convert_tokens_to_ids('<speaker2>')
  mask = []
  i = len(words) - 1
  while words[i] != speaker1 and words[i] != speaker2:
    i -= 1
  fst_token = words[i]

  if fst_token == speaker1:
    fill_token = speaker1
  else:
    fill_token = speaker2

  for i in range(len(words)):
    if words[i] == speaker2 and fill_token == speaker1:
      fill_token = speaker2

    if words[i] == speaker1 and fill_token == speaker2:
      fill_token = speaker1

    if fill_token == speaker1:
      mask.append(0)
    else:
      mask.append(1)
  return mask

def get_example(index):
  history , reply, persona = generate_example(index)
  words, segments, position, sequence = build_inputs([persona], history, reply)
  words = tokenizer(words)['input_ids']

  not_ansewer_tokens_len = len(list(chain(*words[:-len(sequence[-1])])))
  answer_tokens = list(chain(*words[-len(sequence[-1][1:]):]))
  words = list(chain(*words))
  segments = get_segments(words)
  # Prepare our language modeling targets: keep only the reply segment, -1 on the rest
  lm_targets = ([-100] * not_ansewer_tokens_len \
              + [-100] + [-100] + answer_tokens[1:])


  # Store the position of the last tokens for the next-sentence prediction loss
  last_token = len(words) - 1

  input_ids = torch.tensor(words, dtype=torch.long)

  token_type_ids = torch.tensor(segments, dtype=torch.long)


  mc_token_ids = torch.tensor(last_token, dtype=torch.long)


  lm_labels = torch.tensor(lm_targets, dtype=torch.long)
  # Next-sentence prediction labels
  mc_labels = torch.tensor([0], dtype=torch.long)  

  return input_ids, lm_labels, token_type_ids

In [15]:
get_example(1)

(tensor([50258,  1358,   296, 15617,   503,  1140, 16588, 31288,   267,   342,
         46468, 26174,  5218,   530, 19810,   431, 19626,   607,  1358,   404,
          5218,   623,   623,  4130,   404,  5218,   623,   552,   293,   839,
          7035,   276, 50260, 37954,     5,    18,   225, 50261, 37954,    16,
          4182,  1805,    35,    18,   225, 50260,  4925,  9151,    13,  4891,
         26195, 38936,    16,  4866,  1645, 37181,     5,    18,   225, 50261,
          1358,  1215,  1800,  3218,    16,   374,   703, 25634,  9843,    16,
          1514,  6453,   360,   703,  1472,  2764,    16, 34501,  1512,    18,
          5661,  2412,  5001,   334,  1354,    16,   417,  1215, 17635,   498,
           225, 50260, 50262,   618,   417,  1215,   329,  1616,  2533,    16,
         19759,   281,  3369,   758,  2436,  4887,    18,   832,   475,  1749,
          2388,    16,   428,   694, 12133,    18,  9873,  1810,   828,   764,
           694,  2412,  5001,    35,    18,   225, 5

In [16]:
def prepare_input(persona, history):
    sequence = [[bos] + list(chain(*persona))] + history
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens            # position tokens

    print(words)
    words = tokenizer(words)['input_ids']
    words = list(chain(*words))
    input_ids = torch.tensor([words], dtype=torch.long)
    return input_ids

In [17]:
class Dialoges(Dataset):

    def __init__(self, csv_file):
        data = pd.read_csv(csv_file, sep='\t')
        data = data.drop(9635)
        data = data.reset_index(drop=True)
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return get_example(idx)

In [18]:
dataset = Dialoges('drive/MyDrive/dialogues.tsv')

train_loader = DataLoader(dataset,
                          batch_size=1, 
                          shuffle=False)

In [19]:
num_epochs = 3
learning_rate = 0.0000001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
import GPUtil
from tabulate import tabulate

def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train', position=0, leave=True)

    model.train()

    cnt = 0
    for X, y, z in loader:
        X = X.to(device)
        y = y.to(device)
        z = z.to(device)
        
        outputs = model(input_ids = X, labels = X, token_type_ids = z)
        loss = outputs[0]
        losses.append(loss.item())

        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()
       

        if cnt % 1000 == 0 and cnt > 1000:
          save_last_model_path = 'drive/MyDrive/last_model_state_dict.pth'
          save_last_optimizer_path = 'drive/MyDrive/last_optimizer_state_dict.pth'
          torch.save(model.state_dict(), save_last_model_path)
          torch.save(optimizer.state_dict(), save_last_optimizer_path)


        if (cnt % 1000 == 0):
          # GPU information

          print("="*40, "GPU Details", "="*40)
          gpus = GPUtil.getGPUs()
          list_gpus = []
          for gpu in gpus:
              # get the GPU id
              gpu_id = gpu.id
              # name of GPU
              gpu_name = gpu.name
              # get % percentage of GPU usage of that GPU
              gpu_load = f"{gpu.load*100}%"
              # get free memory in MB format
              gpu_free_memory = f"{gpu.memoryFree}MB"
              # get used memory
              gpu_used_memory = f"{gpu.memoryUsed}MB"
              # get total memory
              gpu_total_memory = f"{gpu.memoryTotal}MB"
              # get GPU temperature in Celsius
              gpu_temperature = f"{gpu.temperature} °C"
              gpu_uuid = gpu.uuid
              list_gpus.append((
                  gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
                  gpu_total_memory, gpu_temperature, gpu_uuid
              ))

          print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))
        cnt += 1

    progress_bar.close()
    
    return losses

In [20]:
model.load_state_dict(torch.load('drive/MyDrive/last_model_state_dict.pth'))
optimizer.load_state_dict(torch.load('drive/MyDrive/last_optimizer_state_dict.pth'))

In [21]:
# save_last_model_path = 'drive/MyDrive/last_model_state_dict.pth'
# save_last_optimizer_path = 'drive/MyDrive/last_optimizer_state_dict.pth'
# torch.save(model.state_dict(), save_last_model_path)
# torch.save(optimizer.state_dict(), save_last_optimizer_path)

In [ ]:
train_losses = []
train_perplexities = []

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)
    mean_train_loss = np.mean(epoch_train_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'  
    print(message) 
    # 1160/10012

Train:   0%|          | 1/10012 [00:00<1:04:06,  2.60it/s, loss=2.63, perplexity=13.9]

======================================== GPU Details ========================================
  id  name      load    free memory    used memory    total memory    temperature    uuid
----  --------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla T4  77.0%   6597.0MB       8512.0MB       15109.0MB       68.0 °C        GPU-a4b8efa0-fc05-c268-1845-2815fb769444


Train:  10%|▉         | 1001/10012 [04:29<36:58,  4.06it/s, loss=2.25, perplexity=9.49]

======================================== GPU Details ========================================
  id  name      load    free memory    used memory    total memory    temperature    uuid
----  --------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla T4  93.0%   2267.0MB       12842.0MB      15109.0MB       76.0 °C        GPU-a4b8efa0-fc05-c268-1845-2815fb769444


Train:  20%|█▉        | 2001/10012 [08:56<31:39,  4.22it/s, loss=2.31, perplexity=10.1]

...

======================================== GPU Details ========================================
  id  name      load    free memory    used memory    total memory    temperature    uuid
----  --------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla T4  0.0%    2261.0MB       12848.0MB      15109.0MB       70.0 °C        GPU-a4b8efa0-fc05-c268-1845-2815fb769444


Train:  50%|████▉     | 5001/10012 [26:38<22:15,  3.75it/s, loss=2.32, perplexity=10.2]

======================================== GPU Details ========================================
  id  name      load    free memory    used memory    total memory    temperature    uuid
----  --------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla T4  0.0%    2261.0MB       12848.0MB      15109.0MB       70.0 °C        GPU-a4b8efa0-fc05-c268-1845-2815fb769444


Train:  52%|█████▏    | 5183/10012 [28:51<19:42,  4.08it/s, loss=2.33, perplexity=10.2]

KeyboardInterrupt: ignored

In [156]:
def choose_answer(variants):
  # print(variants.shape)
  for beam_output in variants:
    i = tokenizer.decode(beam_output)
    if i[-1] in '?.!':
      return i

  # print(''.join(i.split('.')[:-1]))
  # print(i)
  return ' '.join(i.split('.')[:-1]) + '.'

In [150]:
def generate_answer(inds, token_type_ids):
  beam_outputs = model.generate(
          input_ids=inds,
          token_type_ids=token_type_ids,
          top_p=0.95, 
          temperature=1,
          max_length= len(inds[0]) + 20, 
          min_length= len(inds[0]) + 5, 
          num_beams=5, 
          no_repeat_ngram_size=2, 
          num_return_sequences=5, 
          early_stopping=True,
          eos_token_id = 50259,
          pad_token_id = 0,
          )

  answer = choose_answer(beam_outputs)
  return answer

In [151]:
def speak():
  
  # 
  about = '<bos> Меня Зовут Анна. мне 20 лет, студнентка. люблю петь. Учусь на дизайнера.'

  history = []
  print('Начните диалог')

  while(True):
    message = input()
    i = 1
    history.append([' <speaker1> ' + message])
    input_text = about + ''.join(list(chain(*history))) + '<speaker2>' + '<fos>' # history[-4:]

    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    token_type_ids = torch.LongTensor(get_segments(tokenizer(input_text, return_tensors='pt')['input_ids'][0])).unsqueeze(0).to(device)

    # print(inds.shape, token_type_ids.shape)
    

    answer = generate_answer(inds, token_type_ids)

    i = 1
    while answer == False:
      answer = generate_answer(inds, token_type_ids)
      print(i)
      i += 1

    history.append([' <speaker2> ' + answer[len(input_text):]])
    # print(inds)
    # print(token_type_ids)
    print('-', answer[len(input_text):])
    # print(history)



# repetition_penalty=5.0,
#           top_k=5, 
#           top_p=0.95, 
#           temperature=1,
#           max_length= len(inds[0]) + 20,
#           do_sample=True,
#           no_repeat_ngram_size=10,
#           pad_token_id = 50259,
#           # length_penalty = 0.5

In [158]:
speak()

Начните диалог
привет! Как дела?
- Привет  Хорошо  А у тебя?  Чем занимаешься?).
Все хорошо, сижу дома. Как тебя зовут?
- Меня зовут Анна, мне 21 год, я студентка 😊. А тебя как?.
Я Ваня. Приятно познакомиться. Аня, ты любишь животных?
- Люблю животных, у меня есть попугай 🤣. Нет, не люблю животных.
Какой породы твой попугай?
- Среднеазиатская овчарка, очень умная и преданная собака, обожаю ее.
как ее зовут?
- Анюта, она очень красивая и умная собака. Я очень люблю свою собаку, а ты?
У меня нет собаки( живу в маленькой квартире.
- 
Какие фильмы ты смотришь?
- 
алло?
-  Я люблю смотреть сериалы, особенно про любовь, но больше всего мне нравятся фильмы про путешествия.
А мне больше нравятся детективы.
-  Я тоже люблю путешествовать, поэтому часто езжу в разные страны, в том числе и в Китай.
Ого, и как там?
-   Очень интересно и познавательно. В Китае очень много интересных мест, которые я обязательно посещаю.
Ты была на Великой Китайской стене?
-  Да, это очень интересное место, там очень

KeyboardInterrupt: ignored

In [ ]:

arr = [len(generate_example(i)[0]) for i in range(2000, 2420)]


In [ ]:
print(np.argmax(arr), max(arr))


In [ ]:
np.mean(arr)

In [ ]:
get_example(1)

In [ ]:
tokenizer.convert_tokens_to_ids('<pad>')

In [ ]:
!tar xvzf drive/MyDrive/filtered_dialogs.jsonl.tar.gz

In [ ]:
import csv
with open('filtered_dialogs.jsonl') as csv_json_franken_file:
    jsonreader = csv.reader(csv_json_franken_file, delimiter=':', quotechar='"')
    for row in jsonreader: # This bit reads in a "row" at a time, until finished
        print(', '.join(row))